In [1]:
import pandas as pd
from siuba import *
import numpy as np

import altair as alt

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

from calitp import to_snakecase

from IPython.display import display, Markdown

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You m

In [2]:
pd.set_option("display.max_columns", 100)


In [3]:
df=pd.read_csv('gs://calitp-analytics-data/data-analyses/5310/5310_org_info_manualfill.csv')

In [4]:
bc = to_snakecase(
    pd.read_excel(
        "gs://calitp-analytics-data/data-analyses/grants/Grant+Projects_3_17_2022.xlsx"
    )
)
bc = bc>>filter(_.funding_program.str.contains('5310'))
bc["organization_name"] = (
    bc["organization_name"]
    .str.replace(",", "")
    .str.replace(".", "")
    .str.replace("/", " ")
    #.str.replace("\\", "")
)

/tmp/ipykernel_1168/4035653396.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [5]:
bc.head(5)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time
125,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Amador Transit,BCG0001035,Operating Assistance,300901,64AO18-00677,30144.0,30144.0,30144.0,0.0,0,Open,NaN,NaN,NaN
126,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Angel View Inc,BCG0000324,Operating Assistance,300901,64AO18-00770,125000.0,125000.0,125000.0,0.0,0,Open,NaN,NaN,NaN
127,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,ARC Imperial Valley,BCG0000988,Operating Assistance,300901,64AO18-00630,193211.0,193211.0,193211.0,0.0,0,Open,NaN,NaN,NaN
128,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,ARC Imperial Valley,BCG0002757,Operating Assistance,300901,64AO18-00630,40000.0,40000.0,40000.0,0.0,0,Open,NaN,NaN,NaN
129,2017,5310 Exp,CA-2017-169 | 0017000186-E,2017,Area 1 Agency on Aging,BCG0000392,Operating Assistance,300901,64AO18-00631,146534.0,146534.0,146534.0,0.0,0,Open,NaN,NaN,NaN


In [6]:
bc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1115 entries, 125 to 1635
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   grant_fiscal_year    1115 non-null   int64  
 1   funding_program      1115 non-null   object 
 2   grant_number         1115 non-null   object 
 3   project_year         1115 non-null   int64  
 4   organization_name    1115 non-null   object 
 5   upin                 1115 non-null   object 
 6   description          1115 non-null   object 
 7   ali                  1115 non-null   object 
 8   contract_number      1089 non-null   object 
 9   allocationamount     1115 non-null   float64
 10  encumbered_amount    1115 non-null   float64
 11  expendedamount       1115 non-null   float64
 12  activebalance        1115 non-null   float64
 13  closedoutbalance     1115 non-null   int64  
 14  project_status       1115 non-null   object 
 15  project_closed_by    0 non-null     

In [7]:
p = pd.merge(bc, df, how='left', on='organization_name')

In [8]:
p['description'] = p['description'].str.strip()

In [9]:
(
    p
    >> group_by(_.organization_name, _.funding_program)
    >> summarize(n=_.description.nunique())
    >> arrange(-_.n)
    >> filter(_.n > 1)
)

,organization_name,funding_program,n
39,Community Bridges Liftline,5310 Trad,6
181,United Cerebral Palsy Association of Greater S...,5310 Trad,6
139,Porterville Sheltered Workshop,5310 Trad,5
11,Asian Community Center of Sacramento Valley In...,5310 Trad,4
21,Catholic Charities of the Diocese of Stockton,5310 Trad,4
...,...,...,...
173,The Center for Independent Living,5310 Trad,2
185,Valley Resource Center,5310 Trad,2
187,Victor Valley Community Services Council,5310 Trad,2
188,Victor Valley Transit Authority,5310 Trad,2


In [10]:
p>>count(_.grant_fiscal_year)

,grant_fiscal_year,n
0,2017,513
1,2019,602


In [11]:
p>>count(_.project_year)

,project_year,n
0,2017,507
1,2018,4
2,2019,604


In [12]:
p>>filter(_.project_year==2018)

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
72,2017,5310 Exp,CA-2017-169 | 0017000186-E,2018,County of Sonoma Human Services Department Adu...,BCG0000526,Operating Assistance,300901,64AO18-00794,266975.0,266975.0,266975.0,0.0,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Independent Agency,NaN,Sonoma,Sonoma,NaN,No Match
73,2017,5310 Exp,CA-2017-169 | 0017000186-E,2018,Desert Access and Mobility Inc,BCG0000752,Operating Assistance,300901,64AO16-00777,125000.0,125000.0,125000.0,0.0,0,Open,NaN,NaN,NaN,NaN,NaN,Desert Access and Mobility Inc,NaN,NaN,Non-Profit Organization,NaN,Palm Springs,Riverside,NaN,Name Match; Airtable
510,2017,5310 Trad,CA-2017-169 | 0017000186-T,2018,County of Sonoma Human Services Department Adu...,BCG0000525,Mobility Management,117L00,64AM18-00764,157532.0,157532.0,157532.0,0.0,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Independent Agency,NaN,Sonoma,Sonoma,NaN,No Match
511,2017,5310 Trad,CA-2017-169 | 0017000186-T,2018,County of Sonoma Human Services Department Adu...,BCG0000598,Purchase Computer Software,114208,64AM18-00764,10000.0,10000.0,10000.0,0.0,0,Open,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Independent Agency,NaN,Sonoma,Sonoma,NaN,No Match


In [13]:
p>>count(_.grant_number)


,grant_number,n
0,CA-2017-169 | 0017000186-E,75
1,CA-2017-169 | 0017000186-T,438
2,CA-2020-244 | 0020000273-E,76
3,CA-2020-244 | 0020000273-T,526


In [14]:
(p>>group_by(_.organization_name, _.grant_fiscal_year)>>count(_.description))

,organization_name,grant_fiscal_year,description,n
0,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus,16
1,ARC Bakersfield,2017,Purchase Replacement Van,4
2,ARC Imperial Valley,2017,Operating Assistance,2
3,ARC Imperial Valley,2019,Buy Van For Svc Expansion,4
4,ARC Imperial Valley,2019,Operating Assistance,2
...,...,...,...,...
353,Wilshire Community Services,2019,Operating Assistance,1
354,Work Training Center,2017,Purchase Radios,1
355,Work Training Center,2017,Purchase Replacement < 30 Ft Bus,6
356,Yolo County Transportation District,2017,Mobility Management,2


In [15]:
p>>group_by(_.organization_name)>>count(_.description)

,organization_name,description,n
0,ARC Bakersfield,Purchase Replacement < 30 Ft Bus,16
1,ARC Bakersfield,Purchase Replacement Van,4
2,ARC Imperial Valley,Buy Van For Svc Expansion,4
3,ARC Imperial Valley,Operating Assistance,4
4,Able Industries,Buy <30-Ft Bus For Expansion,6
...,...,...,...
280,Wilshire Community Services,Operating Assistance,2
281,Work Training Center,Purchase Radios,1
282,Work Training Center,Purchase Replacement < 30 Ft Bus,6
283,Yolo County Transportation District,Mobility Management,2


In [16]:
p.description.value_counts()

Purchase Replacement < 30 Ft Bus                                                                464
Buy <30-Ft Bus For Expansion                                                                    138
Purchase Replacement Van                                                                        122
Operating Assistance                                                                            113
Buy Van For Svc Expansion                                                                        93
Mobility Management                                                                              72
Purchase Computer Hardware                                                                       16
Purchase Radios                                                                                  16
Surveillance/Security                                                                            10
Support Equip/Facilities-Equipment                                                                8


In [17]:
nunique = (
    p
    >> group_by(_.organization_name, _.organization_type)
    >> summarize(nunique_desc=_.description.nunique())
    >> arrange(-_.nunique_desc)
)

In [18]:
nunique

,organization_name,organization_type,nunique_desc
28,Community Bridges Liftline,Non-Profit Organization,7
35,County of Sonoma Human Services Department Adu...,Independent Agency,7
136,United Cerebral Palsy Association of Greater S...,Non-Profit Organization,7
85,Mobility Matters,Non-Profit Organization,6
7,Asian Community Center of Sacramento Valley In...,Non-Profit Organization,5
...,...,...,...
135,United Cerebral Palsy Association Inc of Stani...,Non-Profit Organization,1
137,United Cerebral Palsy of San Luis Obispo County,Non-Profit Organization,1
138,Valley Achievement Center,Non-Profit Organization,1
143,West Valley Community Services,Non-Profit Organization,1


In [19]:
# p_agg = (p.groupby(['organization_name'])
#       .agg({'description': lambda x: x.tolist()})
#       .rename({'description' : 'descriptions'}, axis=1)
#       .reset_index())

In [20]:
# p_agg

In [21]:
description = p>>group_by(_.organization_name)>>count(_.description)

In [22]:
description

,organization_name,description,n
0,ARC Bakersfield,Purchase Replacement < 30 Ft Bus,16
1,ARC Bakersfield,Purchase Replacement Van,4
2,ARC Imperial Valley,Buy Van For Svc Expansion,4
3,ARC Imperial Valley,Operating Assistance,4
4,Able Industries,Buy <30-Ft Bus For Expansion,6
...,...,...,...
280,Wilshire Community Services,Operating Assistance,2
281,Work Training Center,Purchase Radios,1
282,Work Training Center,Purchase Replacement < 30 Ft Bus,6
283,Yolo County Transportation District,Mobility Management,2


In [23]:
description_agg = (description.groupby(['organization_name'])
      .agg({'description': lambda x: x.tolist()})
      .rename({'description' : 'descriptions'}, axis=1)
      .reset_index())

In [24]:
description_agg

,organization_name,descriptions
0,ARC Bakersfield,"[Purchase Replacement < 30 Ft Bus, Purchase Re..."
1,ARC Imperial Valley,"[Buy Van For Svc Expansion, Operating Assistance]"
2,Able Industries,[Buy <30-Ft Bus For Expansion]
3,Alegria Community Living,[Purchase Replacement Van]
4,Amador Transit,"[Mobility Management, Operating Assistance]"
...,...,...
142,Vivalon Inc,"[Operating Assistance, Purchase Replacement < ..."
143,West Valley Community Services,[Operating Assistance]
144,Wilshire Community Services,[Operating Assistance]
145,Work Training Center,"[Purchase Radios, Purchase Replacement < 30 Ft..."


In [25]:
description_agg>>filter(_.organization_name =='Amador Transit')

,organization_name,descriptions
4,Amador Transit,"[Mobility Management, Operating Assistance]"


In [26]:
description_agg['descriptions'] = [', '.join(map(str, l)) for l in description_agg['descriptions']]

In [27]:
description_agg

,organization_name,descriptions
0,ARC Bakersfield,"Purchase Replacement < 30 Ft Bus, Purchase Rep..."
1,ARC Imperial Valley,"Buy Van For Svc Expansion, Operating Assistance"
2,Able Industries,Buy <30-Ft Bus For Expansion
3,Alegria Community Living,Purchase Replacement Van
4,Amador Transit,"Mobility Management, Operating Assistance"
...,...,...
142,Vivalon Inc,"Operating Assistance, Purchase Replacement < 3..."
143,West Valley Community Services,Operating Assistance
144,Wilshire Community Services,Operating Assistance
145,Work Training Center,"Purchase Radios, Purchase Replacement < 30 Ft Bus"


In [28]:
org_counts_descriptions =(pd.merge(description_agg, nunique, how='left', on='organization_name'))

In [29]:
org_counts_descriptions

,organization_name,descriptions,organization_type,nunique_desc
0,ARC Bakersfield,"Purchase Replacement < 30 Ft Bus, Purchase Rep...",Non-Profit Organization,2
1,ARC Imperial Valley,"Buy Van For Svc Expansion, Operating Assistance",Non-Profit Organization,2
2,Able Industries,Buy <30-Ft Bus For Expansion,Non-Profit Organization,1
3,Alegria Community Living,Purchase Replacement Van,Non-Profit Organization,1
4,Amador Transit,"Mobility Management, Operating Assistance",Independent Public Agency or Authority of Tran...,2
...,...,...,...,...
142,Vivalon Inc,"Operating Assistance, Purchase Replacement < 3...",Non-Profit Organization,2
143,West Valley Community Services,Operating Assistance,Non-Profit Organization,1
144,Wilshire Community Services,Operating Assistance,Non-Profit Organization,1
145,Work Training Center,"Purchase Radios, Purchase Replacement < 30 Ft Bus",Non-Profit Organization,2


In [30]:
#org_counts_descriptions.to_csv('org_counts_descriptions.csv')

## Specific Project Types
finding project types for 
* wheelchair accessible vehicles
* on-demand scheduling
* dispatching software platforms

In [31]:
p.description.value_counts()

Purchase Replacement < 30 Ft Bus                                                                464
Buy <30-Ft Bus For Expansion                                                                    138
Purchase Replacement Van                                                                        122
Operating Assistance                                                                            113
Buy Van For Svc Expansion                                                                        93
Mobility Management                                                                              72
Purchase Computer Hardware                                                                       16
Purchase Radios                                                                                  16
Surveillance/Security                                                                            10
Support Equip/Facilities-Equipment                                                                8


In [32]:
p>>filter(_.description.str.contains("Bus"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description
75,Able Industries,2017,Buy <30-Ft Bus For Expansion
76,Able Industries,2017,Buy <30-Ft Bus For Expansion
77,Able Industries,2017,Buy <30-Ft Bus For Expansion
78,Able Industries,2017,Buy <30-Ft Bus For Expansion
79,Able Industries,2017,Buy <30-Ft Bus For Expansion
...,...,...,...
1110,Vivalon Inc,2019,Purchase Replacement < 30 Ft Bus
1111,Vivalon Inc,2019,Purchase Replacement < 30 Ft Bus
1112,Vivalon Inc,2019,Purchase Replacement < 30 Ft Bus
1113,Vivalon Inc,2019,Purchase Replacement < 30 Ft Bus


In [33]:
p>>filter(_.description.str.contains("Bus"))>>count(_.description)>>arrange(-_.n)

,description,n
3,Purchase Replacement < 30 Ft Bus,464
2,Buy <30-Ft Bus For Expansion,138
0,Buy 30-Ft Bus For Expansion,2
1,Buy 35-Ft Bus For Expansion,1
4,Purchase Replacement < 30 Ft Bus for JDA,1


In [34]:
display(Markdown(
    f"There are {len(p>>filter(_.description.str.contains('Bus'))>>count(_.organization_name))} "
    f"organizations with Bus purchases as project types"))

There are 75 organizations with Bus purchases as project types

In [35]:
p>>filter(_.description.str.contains("Van"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )>>count(_.description)

,description,n
0,Buy Van For Svc Expansion,93
1,Operating Assistance - Operating Accessible Vans,1
2,Purchase Replacement Van,122
3,Purchase Replacement Van 1,1
4,Purchase Replacement Van 2,1


In [36]:
(p[(p.description.str.contains("Van")==True) & (p.description.str.contains("Assistance")==False)])>>select(
    _.organization_name,
    _.project_year,
    _.description
)

,organization_name,project_year,description
34,Friends of Children with Special Needs,2017,Buy Van For Svc Expansion
35,Friends of Children with Special Needs,2017,Buy Van For Svc Expansion
93,ARC Bakersfield,2017,Purchase Replacement Van
94,ARC Bakersfield,2017,Purchase Replacement Van
103,ARC Bakersfield,2017,Purchase Replacement Van
...,...,...,...
1039,United Cerebral Palsy Association of Greater S...,2019,Buy Van For Svc Expansion
1089,Valley Resource Center,2019,Purchase Replacement Van
1090,Valley Resource Center,2019,Purchase Replacement Van
1095,Valley Resource Center,2019,Purchase Replacement Van


In [37]:
p[(p.description.str.contains("Van")==True) & (p.description.str.contains("Assistance")==False)]>>count(_.description)

,description,n
0,Buy Van For Svc Expansion,93
1,Purchase Replacement Van,122
2,Purchase Replacement Van 1,1
3,Purchase Replacement Van 2,1


In [38]:
len(p[(p.description.str.contains("Van")==True) & (p.description.str.contains("Assistance")==False)]>>count(_.organization_name))

38

In [39]:
display(Markdown(
    f"There are " 
    f"{len(p[(p.description.str.contains('Van')==True) & (p.description.str.contains('Assistance')==False)]>>count(_.organization_name))} "
    f"organizations with Van purchases as project types"))

There are 38 organizations with Van purchases as project types

In [40]:
p>>filter(_.description.str.contains("On-demand"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description


In [41]:
display(Markdown(
    f"There are " 
    f"{len(p>>filter(_.description.str.contains('On-demand'))>>select(_.organization_name,_.project_year, _.description))} "
    f"organizations with On-Demand purchases as project types"))

There are 0 organizations with On-Demand purchases as project types

In [42]:
p>>filter(_.description.str.contains("Operating Assistance"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description
0,Amador Transit,2017,Operating Assistance
1,Angel View Inc,2017,Operating Assistance
2,ARC Imperial Valley,2017,Operating Assistance
3,ARC Imperial Valley,2017,Operating Assistance
4,Area 1 Agency on Aging,2017,Operating Assistance
...,...,...,...
584,United Cerebral Palsy Association of Greater S...,2019,Operating Assistance
585,Valley Resource Center,2019,Operating Assistance
586,West Valley Community Services,2019,Operating Assistance
587,West Valley Community Services,2019,Operating Assistance


In [43]:
p>>filter(_.description.str.contains("Operating Assistance"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )>>count(_.organization_name)

,organization_name,n
0,ARC Imperial Valley,4
1,Amador Transit,1
2,Angel View Inc,3
3,Area 1 Agency on Aging,1
4,Asian Community Center of Sacramento Valley In...,6
...,...,...
60,Valley Resource Center,2
61,Victor Valley Community Services Council,1
62,Vivalon Inc,1
63,West Valley Community Services,3


In [44]:
display(Markdown(
    f"There are " 
    f"{len((p>>filter(_.description.str.contains('Operating Assistance'))>>select(_.organization_name,_.project_year,_.description))>>count(_.organization_name))} "
    f"organizations with Operating Assistance purchases as project types"))
       

There are 65 organizations with Operating Assistance purchases as project types

In [45]:
p>>filter(_.description.str.contains("Software"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description
117,Asian Community Center of Sacramento Valley In...,2017,Purchase Computer Software
118,Asian Community Center of Sacramento Valley In...,2017,Purchase Computer Software
508,Yolo County Transportation District,2017,Purchase Computer Software
509,Yolo County Transportation District,2017,Purchase Computer Software
511,County of Sonoma Human Services Department Adu...,2018,Purchase Computer Software
631,Catholic Charities of the Diocese of Stockton,2019,Purchase Computer Software
632,Catholic Charities of the Diocese of Stockton,2019,Purchase Computer Software


In [46]:
p>>filter(_.description.str.contains("Hardware"))>>select(_.organization_name,
                                                     _.project_year,
                                                     _.description
                                                    )

,organization_name,project_year,description
111,Asian Community Center of Sacramento Valley In...,2017,Purchase Computer Hardware
166,Community Bridges Liftline,2017,Purchase Computer Hardware
231,Foothill AIDS Project,2017,Purchase Computer Hardware
421,San Joaquin Regional Transit District,2017,Purchase Computer Hardware
477,United Cerebral Palsy Association of Greater S...,2017,Purchase Computer Hardware
478,United Cerebral Palsy Association of Greater S...,2017,Purchase Computer Hardware
629,Catholic Charities of the Diocese of Stockton,2019,Purchase Computer Hardware
630,Catholic Charities of the Diocese of Stockton,2019,Purchase Computer Hardware
653,Community Bridges Liftline,2019,Purchase Computer Hardware
654,Community Bridges Liftline,2019,Purchase Computer Hardware


In [47]:
p.query('description.str.contains("Hardware") or description.str.contains("Software")')>>count(_.description)

,description,n
0,Purchase Computer Hardware,16
1,Purchase Computer Software,7


In [48]:
(p.query('description.str.contains("Hardware") or description.str.contains("Software")')>>count(_.organization_name))

,organization_name,n
0,Asian Community Center of Sacramento Valley In...,3
1,Catholic Charities of the Diocese of Stockton,4
2,Community Bridges Liftline,3
3,County of Sonoma Human Services Department Adu...,1
4,Foothill AIDS Project,1
5,Milestones of Development INC,2
6,Porterville Sheltered Workshop,2
7,San Joaquin Regional Transit District,1
8,Sunline Transit Agency,2
9,United Cerebral Palsy Association of Greater S...,2


In [49]:
display(Markdown(
    f"There are " 
    f"{len((p[(p['description'].str.contains('Hardware')) | (p['description'].str.contains('Software'))])>>count(_.organization_name))} "
    f"organizations with Hardware or Software purchases as project types"))
       

There are 11 organizations with Hardware or Software purchases as project types

In [53]:
#coming back to check to make sure the len is the same
#len(add_cat>>filter(_.hardware_software_purchase==1)>>count(_.organization_name))

#it does

# Description by Year

In [54]:
description_year = p>>group_by(_.organization_name, _.project_year)>>count(_.description)

In [55]:
description_year

,organization_name,project_year,description,n
0,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus,16
1,ARC Bakersfield,2017,Purchase Replacement Van,4
2,ARC Imperial Valley,2017,Operating Assistance,2
3,ARC Imperial Valley,2019,Buy Van For Svc Expansion,4
4,ARC Imperial Valley,2019,Operating Assistance,2
...,...,...,...,...
354,Wilshire Community Services,2019,Operating Assistance,1
355,Work Training Center,2017,Purchase Radios,1
356,Work Training Center,2017,Purchase Replacement < 30 Ft Bus,6
357,Yolo County Transportation District,2017,Mobility Management,2


In [56]:
p_agg2 = (description_year.groupby(['organization_name', 'project_year'])
      .agg({'description': lambda x: x.tolist()})
      .rename({'description' : 'descriptions'},axis=1)
      .reset_index())

In [57]:
p_agg2

,organization_name,project_year,descriptions
0,ARC Bakersfield,2017,"[Purchase Replacement < 30 Ft Bus, Purchase Re..."
1,ARC Imperial Valley,2017,[Operating Assistance]
2,ARC Imperial Valley,2019,"[Buy Van For Svc Expansion, Operating Assistance]"
3,Able Industries,2017,[Buy <30-Ft Bus For Expansion]
4,Alegria Community Living,2019,[Purchase Replacement Van]
...,...,...,...
221,West Valley Community Services,2019,[Operating Assistance]
222,Wilshire Community Services,2017,[Operating Assistance]
223,Wilshire Community Services,2019,[Operating Assistance]
224,Work Training Center,2017,"[Purchase Radios, Purchase Replacement < 30 Ft..."


In [58]:
p_agg2>>filter(_.organization_name =='Amador Transit')

,organization_name,project_year,descriptions
5,Amador Transit,2017,"[Mobility Management, Operating Assistance]"
6,Amador Transit,2019,[Mobility Management]


In [59]:
p_agg2['descriptions'] = [', '.join(map(str, l)) for l in p_agg2['descriptions']]

In [60]:
p_agg2

,organization_name,project_year,descriptions
0,ARC Bakersfield,2017,"Purchase Replacement < 30 Ft Bus, Purchase Rep..."
1,ARC Imperial Valley,2017,Operating Assistance
2,ARC Imperial Valley,2019,"Buy Van For Svc Expansion, Operating Assistance"
3,Able Industries,2017,Buy <30-Ft Bus For Expansion
4,Alegria Community Living,2019,Purchase Replacement Van
...,...,...,...
221,West Valley Community Services,2019,Operating Assistance
222,Wilshire Community Services,2017,Operating Assistance
223,Wilshire Community Services,2019,Operating Assistance
224,Work Training Center,2017,"Purchase Radios, Purchase Replacement < 30 Ft Bus"


In [61]:
# year = (p.groupby(['organization_name','project_year'])
#         .agg({'description': lambda x: x.tolist()})
#         .unstack('project_year', fill_value=0))

In [62]:
# year

In [63]:
# get rid of the duplicates by year

In [64]:
year2 = (description_year.groupby(['organization_name','project_year'])
        .agg({'description': lambda x: x.tolist()})
        #.unstack('project_year', fill_value=0)
        )
         

In [65]:
year2['description'] = [', '.join(map(str, l)) for l in year2['description']]
# year2['2017'] = [', '.join(map(str, l)) for l in year2['2017']]
# year2['2018'] = [', '.join(map(str, l)) for l in year2['2018']]
# year2['2019'] = [', '.join(map(str, l)) for l in year2['2019']]

In [66]:
year2 = year2.unstack('project_year', fill_value=0)

In [67]:
year2

description  \
project_year                                                                      2017   
organization_name                                                                        
ARC Bakersfield                      Purchase Replacement < 30 Ft Bus, Purchase Rep...   
ARC Imperial Valley                                               Operating Assistance   
Able Industries                                           Buy <30-Ft Bus For Expansion   
Alegria Community Living                                                             0   
Amador Transit                               Mobility Management, Operating Assistance   
...                                                                                ...   
Vivalon Inc                          Operating Assistance, Purchase Replacement < 3...   
West Valley Community Services                                                       0   
Wilshire Community Services                                       Operating Assistance   
Work Training Center                 Purchase Radios, Purchase Replacement < 30 Ft Bus   
Yolo County Transportation District    Mobility Management, Purchase Computer Software   

                                          \
project_year                        2018   
organization_name                          
ARC Bakersfield                        0   
ARC Imperial Valley                    0   
Able Industries                        0   
Alegria Community Living               0   
Amador Transit                         0   
...                                  ...   
Vivalon Inc                            0   
West Valley Community Services         0   
Wilshire Community Services            0   
Work Training Center                   0   
Yolo County Transportation District    0   

                                                                                      
project_year                                                                    2019  
organization_name                                                                     
ARC Bakersfield                                                                    0  
ARC Imperial Valley                  Buy Van For Svc Expansion, Operating Assistance  
Able Industries                                                                    0  
Alegria Community Living                                    Purchase Replacement Van  
Amador Transit                                                   Mobility Management  
...                                                                              ...  
Vivalon Inc                                         Purchase Replacement < 30 Ft Bus  
West Valley Community Services                                  Operating Assistance  
Wilshire Community Services                                     Operating Assistance  
Work Training Center                                                               0  
Yolo County Transportation District                                                0  

[147 rows x 3 columns]

In [68]:
#year2.to_csv('project_descitption_by_year.csv')

In [69]:
# testing for single organization: Amador Transit
(p>>filter(_.organization_name =='Amador Transit'))>>select(_.project_year, _.description)

,project_year,description
0,2017,Operating Assistance
81,2017,Mobility Management
599,2019,Mobility Management
600,2019,Mobility Management


In [70]:
# great, exporting
#year.to_csv('by_years.csv')

In [71]:
p_agg2

,organization_name,project_year,descriptions
0,ARC Bakersfield,2017,"Purchase Replacement < 30 Ft Bus, Purchase Rep..."
1,ARC Imperial Valley,2017,Operating Assistance
2,ARC Imperial Valley,2019,"Buy Van For Svc Expansion, Operating Assistance"
3,Able Industries,2017,Buy <30-Ft Bus For Expansion
4,Alegria Community Living,2019,Purchase Replacement Van
...,...,...,...
221,West Valley Community Services,2019,Operating Assistance
222,Wilshire Community Services,2017,Operating Assistance
223,Wilshire Community Services,2019,Operating Assistance
224,Work Training Center,2017,"Purchase Radios, Purchase Replacement < 30 Ft Bus"


## Function for description year

In [72]:
description_year.sample(2)

,organization_name,project_year,description,n
311,Tehama County Transit Agency,2019,Operating Assistance - Paratransit,1
41,Catholic Charities of the Diocese of Stockton,2019,Purchase Computer Software,2


In [73]:
description_year['description'] = description_year['description'].astype(str)

In [74]:
description_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359 entries, 0 to 358
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   organization_name  359 non-null    object
 1   project_year       359 non-null    int64 
 2   description        359 non-null    object
 3   n                  359 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 11.3+ KB


### Using Boolean

In [75]:
description_year.description.value_counts()

Operating Assistance                                                                            87
Purchase Replacement < 30 Ft Bus                                                                81
Mobility Management                                                                             39
Purchase Replacement Van                                                                        30
Buy <30-Ft Bus For Expansion                                                                    28
Buy Van For Svc Expansion                                                                       22
Purchase Computer Hardware                                                                      10
Purchase Radios                                                                                 10
Surveillance/Security                                                                            5
Purchase Computer Software                                                                       4
Support Eq

In [76]:
description_year>> filter(_.description.str.contains('schedul'))>>count(_.description)

,description,n


In [77]:
VEHICLE_PURCHASE = ['bus', 'replacement van','vehicle', 'buy van']
OPERATING_ASSISTANCE = ['operating assistance']
MOBILITY_MANAGEMENT = ['mobility management']
HARDWARE_SOFTWARE_PURCHASE = ['hardware', 'software']
COMMUNICATIONS = ['radios', 'communications']
FACILITIES_EQP =['facilities']
SUBSIDIES = ['subsidy']
SURVEILLANCE=['security']

def categorize_project_descriptions(row):
    
    project_description = row.description.lower()
    

    vehicle_purchase = 0
    operating_assistance = 0
    mobility_management = 0
    hardware_software_purchase = 0
    communications = 0
    facilities_eqp = 0
    subsidies = 0
    surveillance = 0 
        
    if any(word in project_description for word in VEHICLE_PURCHASE):
        vehicle_purchase = 1
        
    if any(word in project_description for word in OPERATING_ASSISTANCE):
        operating_assistance = 1
        
    if any(word in project_description for word in MOBILITY_MANAGEMENT):
        mobility_management = 1
        
    if any(word in project_description for word in HARDWARE_SOFTWARE_PURCHASE):
        hardware_software_purchase = 1
    
    if any(word in project_description for word in COMMUNICATIONS):
        communications = 1
    
    if any(word in project_description for word in FACILITIES_EQP):
        facilities_eqp = 1
        
    if any(word in project_description for word in SUBSIDIES):
        subsidies = 1
        
    if any(word in project_description for word in SURVEILLANCE):
        surveillance = 1
    
    return pd.Series(
        [vehicle_purchase, operating_assistance, mobility_management, hardware_software_purchase, communications, facilities_eqp, surveillance, subsidies], 
        index=['vehicle_purchase', 'operating_assistance', 'mobility_management', 'hardware_software_purchase','communications', 'facilities_eqp',
              'surveillance', 'subsidy']
    )
   

In [78]:
project_categories = description_year.apply(categorize_project_descriptions, axis=1)
add_cat = pd.concat([description_year, project_categories], axis=1)
work_cols = list(project_categories.columns)
add_cat = add_cat.assign(
    project_categories = add_cat[work_cols].sum(axis=1)
)

In [79]:
add_cat.sample(5)

,organization_name,project_year,description,n,vehicle_purchase,operating_assistance,mobility_management,hardware_software_purchase,communications,facilities_eqp,surveillance,subsidy,project_categories
268,Pride Industries One Inc,2017,Buy <30-Ft Bus For Expansion,6,1,0,0,0,0,0,0,0,1
295,San Luis Obispo Council of Governments,2019,Mobility Management,2,0,0,1,0,0,0,0,0,1
239,Omnitrans,2019,Purchase Replacement < 30 Ft Bus,6,1,0,0,0,0,0,0,0,1
110,Drivers for Survivors Inc,2017,Operating Assistance,1,0,1,0,0,0,0,0,0,1
20,Asian Community Center of Sacramento Valley In...,2019,Operating Assistance,3,0,1,0,0,0,0,0,0,1


In [80]:
add_cat.project_categories.value_counts()

1    359
Name: project_categories, dtype: int64

In [81]:
add_cat>>filter(_.project_categories==0)>>count(_.description)>>arrange(-_.n)

,description,n


### Searching using lambda

In [82]:
p_agg2[p_agg2['descriptions'].apply(lambda x: 'Buy Van' in x)]

,organization_name,project_year,descriptions
2,ARC Imperial Valley,2019,"Buy Van For Svc Expansion, Operating Assistance"
10,Asian Community Center of Sacramento Valley In...,2017,"Buy <30-Ft Bus For Expansion, Buy Van For Svc ..."
22,Catholic Charities of the Diocese of Stockton,2017,"Buy Van For Svc Expansion, Operating Assistance"
23,Catholic Charities of the Diocese of Stockton,2019,"Buy Van For Svc Expansion, Mobility Management..."
40,Common Ground Senior Services,2017,"Buy Van For Svc Expansion, Mobility Management"
48,Contra Costa ARC,2017,"Buy Van For Svc Expansion, One vehicle for ser..."
50,County of Nevada Public Works Transit Services...,2019,"Buy <30-Ft Bus For Expansion, Buy Van For Svc ..."
82,Friends of Children with Special Needs,2017,"Buy Van For Svc Expansion, Operating Assistance"
83,Friends of Children with Special Needs,2019,"Buy Van For Svc Expansion, Mobility Management"
86,Futures Explored Inc,2017,"Buy Van For Svc Expansion, Purchase Replacemen..."


In [86]:
p_agg2[p_agg2['descriptions'].apply(lambda x: 'Operating Assistance' in x)]


,organization_name,project_year,descriptions
1,ARC Imperial Valley,2017,Operating Assistance
2,ARC Imperial Valley,2019,"Buy Van For Svc Expansion, Operating Assistance"
5,Amador Transit,2017,"Mobility Management, Operating Assistance"
7,Angel View Inc,2017,"Buy <30-Ft Bus For Expansion, Operating Assist..."
8,Angel View Inc,2019,"Operating Assistance, Purchase Replacement < 3..."
...,...,...,...
215,Victor Valley Community Services Council,2017,"Buy 35-Ft Bus For Expansion, Operating Assistance"
219,Vivalon Inc,2017,"Operating Assistance, Purchase Replacement < 3..."
221,West Valley Community Services,2019,Operating Assistance
222,Wilshire Community Services,2017,Operating Assistance


In [87]:
p_agg2

,organization_name,project_year,descriptions
0,ARC Bakersfield,2017,"Purchase Replacement < 30 Ft Bus, Purchase Rep..."
1,ARC Imperial Valley,2017,Operating Assistance
2,ARC Imperial Valley,2019,"Buy Van For Svc Expansion, Operating Assistance"
3,Able Industries,2017,Buy <30-Ft Bus For Expansion
4,Alegria Community Living,2019,Purchase Replacement Van
...,...,...,...
221,West Valley Community Services,2019,Operating Assistance
222,Wilshire Community Services,2017,Operating Assistance
223,Wilshire Community Services,2019,Operating Assistance
224,Work Training Center,2017,"Purchase Radios, Purchase Replacement < 30 Ft Bus"


## Project Type Analysis

### by organization and year

In [88]:
(p
 >> group_by(_.organization_name, _.project_year)
 >> summarize(sum_allocated_by_year = _.allocationamount.sum())
)

,organization_name,project_year,sum_allocated_by_year
0,ARC Bakersfield,2017,1008340.28
1,ARC Imperial Valley,2017,233211.00
2,ARC Imperial Valley,2019,315967.00
3,Able Industries,2017,176258.13
4,Alegria Community Living,2019,280000.00
...,...,...,...
221,West Valley Community Services,2019,649453.00
222,Wilshire Community Services,2017,60000.00
223,Wilshire Community Services,2019,150000.00
224,Work Training Center,2017,443352.00


### by organization, year, and description

In [89]:
project_funds = (p
 >> group_by(_.organization_name, _.project_year, _.description)
 >> summarize(sum_allocated = _.allocationamount.sum())
)

In [90]:
project_funds.head(5)

,organization_name,project_year,description,sum_allocated
0,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus,812340.28
1,ARC Bakersfield,2017,Purchase Replacement Van,196000.00
2,ARC Imperial Valley,2017,Operating Assistance,233211.00
3,ARC Imperial Valley,2019,Buy Van For Svc Expansion,112000.00
4,ARC Imperial Valley,2019,Operating Assistance,203967.00


In [97]:
project_funds2 = pd.merge(
    project_funds, p_agg2, how="left", on="organization_name"
) >> select(_.organization_name, _.project_year_x, _.description, _.sum_allocated)

In [98]:
project_funds2

,organization_name,project_year_x,description,sum_allocated
0,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus,812340.28
1,ARC Bakersfield,2017,Purchase Replacement Van,196000.00
2,ARC Imperial Valley,2017,Operating Assistance,233211.00
3,ARC Imperial Valley,2017,Operating Assistance,233211.00
4,ARC Imperial Valley,2019,Buy Van For Svc Expansion,112000.00
...,...,...,...,...
624,Wilshire Community Services,2019,Operating Assistance,150000.00
625,Work Training Center,2017,Purchase Radios,5352.00
626,Work Training Center,2017,Purchase Replacement < 30 Ft Bus,438000.00
627,Yolo County Transportation District,2017,Mobility Management,0.00


In [105]:
project_funds2 = (project_funds2.drop_duplicates())

In [109]:
# should be the same entries.

In [110]:
project_funds>>filter(_.organization_name=='ARC Imperial Valley')

,organization_name,project_year,description,sum_allocated
2,ARC Imperial Valley,2017,Operating Assistance,233211.0
3,ARC Imperial Valley,2019,Buy Van For Svc Expansion,112000.0
4,ARC Imperial Valley,2019,Operating Assistance,203967.0


In [111]:
project_funds2>>filter(_.organization_name=='ARC Imperial Valley')

,organization_name,project_year_x,description,sum_allocated
2,ARC Imperial Valley,2017,Operating Assistance,233211.0
4,ARC Imperial Valley,2019,Buy Van For Svc Expansion,112000.0
6,ARC Imperial Valley,2019,Operating Assistance,203967.0


In [112]:
project_funds2

,organization_name,project_year_x,description,sum_allocated
0,ARC Bakersfield,2017,Purchase Replacement < 30 Ft Bus,812340.28
1,ARC Bakersfield,2017,Purchase Replacement Van,196000.00
2,ARC Imperial Valley,2017,Operating Assistance,233211.00
4,ARC Imperial Valley,2019,Buy Van For Svc Expansion,112000.00
6,ARC Imperial Valley,2019,Operating Assistance,203967.00
...,...,...,...,...
623,Wilshire Community Services,2019,Operating Assistance,150000.00
625,Work Training Center,2017,Purchase Radios,5352.00
626,Work Training Center,2017,Purchase Replacement < 30 Ft Bus,438000.00
627,Yolo County Transportation District,2017,Mobility Management,0.00


In [91]:
(p
 >> group_by(_.organization_name, _.project_year, _.description)
 >> summarize(sum_allocated = _.allocationamount.sum())
)>>filter(_.sum_allocated==0)

,organization_name,project_year,description,sum_allocated
19,Asian Community Center of Sacramento Valley In...,2017,Purchase Computer Software,0.0
28,Calaveras County Public Works,2017,Purchase Replacement < 30 Ft Bus,0.0
186,Livermore Amador Valley Transit Authority,2017,Mobility Management-Concord UA,0.0
270,Pride Industries One Inc,2017,Purchase Replacement Van,0.0
297,Santa Clara Valley Transportation Authority,2017,Mobility Management,0.0
357,Yolo County Transportation District,2017,Mobility Management,0.0
358,Yolo County Transportation District,2017,Purchase Computer Software,0.0


### by county

In [92]:
p.sample()

,grant_fiscal_year,funding_program,grant_number,project_year,organization_name,upin,description,ali,contract_number,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,project_status,project_closed_by,project_closed_date,project_closed_time,ntd_id,itp_id,name_NTD_Airtable,doing_business_as,reporter_acronym,organization_type,reporter_type,city,county,mobility_services_operated,merge_status
121,2017,5310 Trad,CA-2017-169 | 0017000186-T,2017,Avenidas,BCG0000667,Purchase Replacement Van,111215,64AC18-00695,49000.0,49000.0,47145.18,1854.82,0,Open,NaN,NaN,NaN,NaN,401.0,Avenidas,NaN,NaN,Non-Profit Organization,NaN,Palo Alto,Santa Clara,Avenidas Door-to-Door,"Manual Match, Airtable"


In [93]:
(p
 >> group_by(_.county)
 >> summarize(sum_allocated_by_county = _.allocationamount.sum())
)

,county,sum_allocated_by_county
0,Alameda,2982664.00
1,"Alameda, Contra Costa",671900.00
2,Amador,469995.00
3,"Amador, Calaveras",299479.40
4,Butte,1235790.10
5,"Butte, Glen",813194.00
6,Calaveras,0.00
7,"Calaveras, Amador, Tuolumne",826157.00
8,Colusa,162400.00
9,Contra Costa,3007198.00


### by project type

In [94]:
(p
 >> group_by(_.description)
 >> summarize(sum_allocated_by_pt = _.allocationamount.sum())
)

,description,sum_allocated_by_pt
0,2021 Operating Assistance for JDA,326400.00
1,211 One Stop Call Center & Travel Training - M...,457453.00
2,5310 Lake County NEMT Operating Assistance,225139.00
3,5310 Lake County Volunteer Driver Program Oper...,65520.00
4,Bay Area Mobility Management and Coordination ...,208687.00
5,Buy 30-Ft Bus For Expansion,126000.00
6,Buy 35-Ft Bus For Expansion,93000.00
7,Buy <30-Ft Bus For Expansion,5492578.87
8,Buy Van For Svc Expansion,3159015.42
9,DriveCam (Support Equip/Facilities-Equipment),1025.00
